In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Case Study: AI-Powered Skin Lesion Triage with Vision Transformers
## Implementation Notebook

Welcome to the implementation notebook for the SkinSight AI case study. In this notebook, you will build a Vision Transformer (ViT) model for classifying dermoscopic skin lesion images into 8 diagnostic categories.

**What you will build:**
- A data pipeline for the ISIC 2019 skin lesion dataset
- A CNN baseline (ResNet-18) for comparison
- A Vision Transformer from scratch, understanding each component
- A full training and evaluation pipeline
- Inference benchmarking for production readiness
- An ethical impact assessment for clinical deployment

**Prerequisites:** Familiarity with PyTorch, convolutional neural networks, and the Transformer architecture. Read the accompanying case study document (Sections 1-2) before starting.

**Runtime:** This notebook is designed to run on Google Colab with a T4 GPU. Total runtime is approximately 60-90 minutes.

---

## 3.1 Data Acquisition Strategy

We use the **ISIC 2019 Skin Lesion Classification Challenge** dataset, containing 25,331 dermoscopic images across 8 diagnostic categories. This is a real clinical dataset used in international dermatology AI competitions.

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/vision-transformers-from-scratch/practice/0/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
# Setup and imports
!pip install -q datasets transformers timm torch torchvision matplotlib seaborn scikit-learn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import torchvision.transforms as transforms
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    balanced_accuracy_score, roc_curve
)
from collections import Counter
import time
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Class names for the ISIC 2019 dataset
CLASS_NAMES = [
    "Melanoma", "Melanocytic nevus", "Basal cell carcinoma",
    "Actinic keratosis", "Benign keratosis", "Dermatofibroma",
    "Vascular lesion", "Squamous cell carcinoma"
]
NUM_CLASSES = len(CLASS_NAMES)

# Load the ISIC 2019 dataset from Hugging Face
# This may take a few minutes on first run (downloads ~9GB)
print("Loading ISIC 2019 dataset...")
dataset = load_dataset("marmal88/skin_cancer", split="train")
print(f"Total samples: {len(dataset)}")
print(f"Features: {dataset.features}")

### TODO: Data Augmentation Pipeline

Dermoscopic images have no canonical orientation -- a lesion can appear at any angle. This means we can apply aggressive geometric augmentations (rotation, flips) without changing the diagnostic label. We also apply color jitter to simulate variations in lighting and camera settings.

In [ ]:
def create_transforms(img_size=224):
    """
    Create training and evaluation image transforms for dermoscopic images.

    Training transforms should include:
    1. Resize to (img_size + 32) x (img_size + 32) for random cropping headroom
    2. RandomResizedCrop to img_size x img_size (scale 0.8 to 1.0)
    3. RandomHorizontalFlip (p=0.5)
    4. RandomVerticalFlip (p=0.5) -- lesions have no canonical orientation
    5. RandomRotation (0 to 360 degrees) -- lesions can appear at any angle
    6. ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    7. ToTensor()
    8. Normalize with ImageNet mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]

    Evaluation transforms should include:
    1. Resize to (img_size + 32) x (img_size + 32)
    2. CenterCrop to img_size x img_size
    3. ToTensor()
    4. Normalize with ImageNet statistics

    Hints:
    - Use torchvision.transforms.Compose to chain transforms
    - RandomRotation with expand=False keeps the image size constant
    - ColorJitter simulates variations in lighting and camera settings during capture
    - Dermoscopic images can be flipped and rotated without changing their diagnosis

    Args:
        img_size: Target image size (default 224 for ViT-Base)

    Returns:
        train_transform: Composed transform for training
        eval_transform: Composed transform for evaluation
    """
    # TODO: Implement training transforms
    train_transform = None  # Your code here

    # TODO: Implement evaluation transforms
    eval_transform = None  # Your code here

    return train_transform, eval_transform

# Verification
train_tf, eval_tf = create_transforms()
assert train_tf is not None, "Training transform not implemented"
assert eval_tf is not None, "Evaluation transform not implemented"
print("Transforms created successfully.")

### Dataset Wrapper

In [ ]:
class ISICSkinDataset(Dataset):
    """PyTorch Dataset wrapper for the ISIC skin lesion dataset."""

    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item["image"].convert("RGB")
        label = item["dx"]  # diagnostic label (integer)

        if self.transform:
            image = self.transform(image)

        return image, label

### TODO: Weighted Sampler for Class Imbalance

The ISIC dataset is severely imbalanced (melanocytic nevus accounts for ~45% of images, while dermatofibroma is less than 2%). Without mitigation, a model could achieve high accuracy by simply predicting the majority class. A weighted sampler ensures that each training batch contains a balanced representation of all classes.

In [ ]:
def create_weighted_sampler(dataset_labels):
    """
    Create a WeightedRandomSampler to handle class imbalance.

    Steps:
    1. Count the frequency of each class in dataset_labels
    2. Compute a weight for each class: weight_c = 1.0 / count_c
    3. Assign each sample the weight of its class
    4. Create a WeightedRandomSampler with these per-sample weights

    Hints:
    - Use collections.Counter to count class frequencies
    - The sampler's num_samples should equal len(dataset_labels) for one full epoch
    - replacement=True allows oversampling of rare classes

    Args:
        dataset_labels: List or array of integer class labels for all samples

    Returns:
        sampler: torch.utils.data.WeightedRandomSampler
    """
    # TODO: Implement weighted sampler
    # Step 1: Count class frequencies

    # Step 2: Compute per-class weights

    # Step 3: Assign per-sample weights

    # Step 4: Create and return WeightedRandomSampler

    pass

# Verification will happen after dataset split

### Split the Dataset

In [ ]:
# Split the dataset into train (80%) and validation (20%)
from torch.utils.data import random_split

train_tf, eval_tf = create_transforms()

full_dataset = ISICSkinDataset(dataset, transform=None)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Use generator for reproducibility
generator = torch.Generator().manual_seed(42)
train_indices, val_indices = random_split(
    range(len(full_dataset)), [train_size, val_size], generator=generator
)

# Create datasets with appropriate transforms
train_dataset = ISICSkinDataset(
    dataset.select(train_indices.indices), transform=train_tf
)
val_dataset = ISICSkinDataset(
    dataset.select(val_indices.indices), transform=eval_tf
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

# Get training labels for the weighted sampler
train_labels = [dataset[i]["dx"] for i in train_indices.indices]
sampler = create_weighted_sampler(train_labels)

# Create DataLoaders
BATCH_SIZE = 64
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True
)

print(f"Training batches per epoch: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

---

## 3.2 Exploratory Data Analysis

Before building any model, we need to understand the data. The class distribution, image characteristics, and visual features will inform our modeling decisions.

In [ ]:
# Compute class distribution
all_labels = [item["dx"] for item in dataset]
label_counts = Counter(all_labels)

print("Class Distribution:")
print("-" * 50)
for idx, name in enumerate(CLASS_NAMES):
    count = label_counts.get(idx, 0)
    pct = 100.0 * count / len(all_labels)
    print(f"  {name:30s}: {count:5d} ({pct:5.1f}%)")

### TODO: Visualize Class Distribution

In [ ]:
def plot_class_distribution(label_counts, class_names):
    """
    Create a horizontal bar chart showing the class distribution.

    Requirements:
    1. Plot horizontal bars for each class, sorted by count (descending)
    2. Color malignant classes (Melanoma, BCC, AK, SCC) in red/orange
       and benign classes (NV, BKL, DF, VASC) in blue/green
    3. Add count labels at the end of each bar
    4. Add a title: "ISIC 2019 Class Distribution"
    5. Label axes appropriately

    Hints:
    - Use plt.barh() for horizontal bars
    - Malignant class indices: [0, 2, 3, 7] (MEL, BCC, AK, SCC)
    - Use different colors for malignant vs benign to highlight the clinical grouping
    - The extreme imbalance should be visually obvious

    Args:
        label_counts: Counter object with class index -> count
        class_names: List of class name strings
    """
    # TODO: Implement the class distribution plot
    pass

plot_class_distribution(label_counts, CLASS_NAMES)

### TODO: Visualize Sample Images

In [ ]:
def plot_sample_images(dataset, class_names, samples_per_class=3):
    """
    Display a grid of sample images from each class.

    Requirements:
    1. Create a grid with NUM_CLASSES rows and samples_per_class columns
    2. For each class, randomly select samples_per_class images
    3. Display each image with its class name as the row label
    4. Set figure size to (samples_per_class * 3, NUM_CLASSES * 3)

    Hints:
    - Use plt.subplots(NUM_CLASSES, samples_per_class)
    - Filter dataset indices by class label to find samples of each class
    - Display raw images (no normalization) for visual inspection

    Args:
        dataset: The Hugging Face dataset object
        class_names: List of class name strings
        samples_per_class: Number of samples to show per class
    """
    # TODO: Implement the sample image grid
    pass

plot_sample_images(dataset, CLASS_NAMES)

### Thought Questions

1. How severe is the class imbalance? What is the ratio between the most common and rarest class? How might this affect model training without any mitigation?
2. Looking at the sample images, what visual features distinguish melanoma from benign nevi? Can you identify features that would require global context (comparing distant regions of the image) versus purely local features?
3. If you trained a model that always predicted the most common class, what would its accuracy be? What would its balanced accuracy be? Why is balanced accuracy a better metric for this dataset?

---

## 3.3 Baseline Approach

We establish a CNN baseline using ResNet-18, a well-understood convolutional architecture. This gives us a performance floor and helps us quantify how much the Vision Transformer improves.

### TODO: Create a CNN Baseline

In [ ]:
import timm

def create_baseline_model(num_classes=8, pretrained=True):
    """
    Create a pretrained ResNet-18 baseline model for skin lesion classification.

    Steps:
    1. Load a pretrained ResNet-18 using the timm library
    2. Replace the classification head with a new Linear layer for num_classes
    3. Move the model to the appropriate device

    Hints:
    - Use timm.create_model('resnet18', pretrained=True, num_classes=num_classes)
    - timm handles the head replacement automatically when you specify num_classes
    - This model has ~11M parameters (much smaller than ViT-Base's 86M)

    Args:
        num_classes: Number of output classes
        pretrained: Whether to use ImageNet-pretrained weights

    Returns:
        model: The ResNet-18 model on the correct device
    """
    # TODO: Create and return the baseline model
    pass

baseline_model = create_baseline_model()
total_params = sum(p.numel() for p in baseline_model.parameters())
print(f"Baseline parameters: {total_params / 1e6:.1f}M")

### TODO: Training and Evaluation Functions

These functions will be reused for both the baseline and the ViT model.

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    """
    Train the model for one epoch.

    Steps:
    1. Set model to training mode
    2. Iterate over batches in the dataloader
    3. For each batch: forward pass, compute loss, backward pass, optimizer step
    4. Track running loss and correct predictions

    Hints:
    - Remember to zero gradients before each backward pass
    - Use torch.argmax(outputs, dim=1) to get predicted classes
    - Return average loss and accuracy for the epoch

    Args:
        model: The neural network model
        dataloader: Training DataLoader
        criterion: Loss function
        optimizer: Optimizer
        device: torch device

    Returns:
        avg_loss: Average loss over the epoch
        accuracy: Training accuracy (correct / total)
    """
    # TODO: Implement the training loop for one epoch
    pass


def evaluate(model, dataloader, criterion, device):
    """
    Evaluate the model on a dataset.

    Steps:
    1. Set model to evaluation mode
    2. Disable gradient computation (torch.no_grad)
    3. Iterate over batches, collecting predictions and true labels
    4. Compute loss, accuracy, and balanced accuracy

    Hints:
    - Use torch.no_grad() context manager for efficiency
    - Collect all predictions and labels for computing balanced accuracy
    - Use sklearn.metrics.balanced_accuracy_score

    Args:
        model: The neural network model
        dataloader: Evaluation DataLoader
        criterion: Loss function
        device: torch device

    Returns:
        avg_loss: Average loss
        accuracy: Overall accuracy
        balanced_acc: Balanced accuracy across classes
        all_preds: List of all predicted class indices
        all_labels: List of all true class labels
    """
    # TODO: Implement the evaluation function
    pass

### Train the Baseline

In [ ]:
# Train baseline for 5 epochs (quick training to establish a floor)
# Set up the training pipeline:
# 1. Define criterion: CrossEntropyLoss with class weights
# 2. Define optimizer: AdamW with lr=1e-4, weight_decay=0.01
# 3. Train for 5 epochs, printing train loss, val loss, and val balanced accuracy

def compute_class_weights(labels, num_classes=8):
    """
    Compute inverse-frequency class weights for weighted cross-entropy loss.
    Formula: w_c = N_total / (num_classes * N_c)
    """
    counts = Counter(labels)
    total = len(labels)
    weights = torch.FloatTensor([
        total / (num_classes * counts.get(i, 1)) for i in range(num_classes)
    ])
    return weights.to(device)

class_weights = compute_class_weights(train_labels)
print("Class weights:", {CLASS_NAMES[i]: f"{w:.2f}" for i, w in enumerate(class_weights)})

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(baseline_model.parameters(), lr=1e-4, weight_decay=0.01)

print("\nTraining ResNet-18 Baseline (5 epochs)...")
print("-" * 65)
baseline_history = {'train_loss': [], 'val_loss': [], 'val_balanced_acc': []}

for epoch in range(5):
    train_loss, train_acc = train_one_epoch(
        baseline_model, train_loader, criterion, optimizer, device
    )
    val_loss, val_acc, val_bal_acc, val_preds, val_labels = evaluate(
        baseline_model, val_loader, criterion, device
    )
    baseline_history['train_loss'].append(train_loss)
    baseline_history['val_loss'].append(val_loss)
    baseline_history['val_balanced_acc'].append(val_bal_acc)
    print(f"Epoch {epoch+1}/5 | Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Balanced Acc: {val_bal_acc:.4f}")

print(f"\nBaseline best balanced accuracy: {max(baseline_history['val_balanced_acc']):.4f}")

---

## 3.4 Model Design: Vision Transformer

Now we build the Vision Transformer from scratch. Each component connects back to the mathematical foundations described in the article.

Recall the three key stages:
1. **Patch Embedding**: Divide image into 16x16 patches, flatten, project to D dimensions, add [CLS] token and position embeddings
2. **Transformer Encoder**: Stack of L blocks, each with Multi-Head Self-Attention + MLP, both with pre-norm and residual connections
3. **Classification Head**: Extract [CLS] token, apply LayerNorm, project to num_classes

### TODO: Implement Patch Embedding

The core insight of ViT: reshape an image into a sequence of patches and treat each patch as a token, just like words in a sentence.

In [ ]:
class PatchEmbedding(nn.Module):
    """
    Convert an image into a sequence of patch embeddings.

    Architecture:
    1. Use nn.Conv2d with kernel_size=patch_size and stride=patch_size to extract
       and project patches in one operation (equivalent to flatten + linear projection)
    2. Create a learnable [CLS] token parameter of shape (1, 1, embed_dim)
    3. Create learnable position embeddings of shape (1, num_patches + 1, embed_dim)

    Forward pass:
    1. Apply the Conv2d projection: (B, 3, 224, 224) -> (B, embed_dim, 14, 14)
    2. Flatten spatial dims and transpose: -> (B, 196, embed_dim)
    3. Expand [CLS] token to batch size and prepend: -> (B, 197, embed_dim)
    4. Add position embeddings
    5. Return the sequence of 197 embedded tokens

    Hints:
    - nn.Conv2d(in_channels, embed_dim, kernel_size=P, stride=P) extracts PxP patches
    - Use .flatten(2).transpose(1, 2) to reshape Conv2d output to (B, N, D)
    - Initialize [CLS] token and position embeddings with nn.Parameter(torch.randn(...))
    - The [CLS] token must be expanded to match the batch dimension using .expand(B, -1, -1)
    """

    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
        super().__init__()
        self.num_patches = (img_size // patch_size) ** 2

        # TODO: Implement the three components:
        # 1. self.projection = nn.Conv2d(...)
        # 2. self.cls_token = nn.Parameter(...)
        # 3. self.position_embeddings = nn.Parameter(...)

    def forward(self, x):
        B = x.shape[0]

        # TODO: Implement the forward pass
        # 1. Project patches
        # 2. Reshape to (B, num_patches, embed_dim)
        # 3. Prepend [CLS] token
        # 4. Add position embeddings
        # 5. Return

        pass

# Verification
patch_embed = PatchEmbedding().to(device)
test_input = torch.randn(2, 3, 224, 224).to(device)
test_output = patch_embed(test_input)
assert test_output.shape == (2, 197, 768), f"Expected (2, 197, 768), got {test_output.shape}"
print(f"PatchEmbedding output shape: {test_output.shape} -- correct!")

### TODO: Implement a Transformer Block

Each block performs two operations with residual connections:
- Multi-Head Self-Attention: every patch attends to every other patch
- MLP: two-layer feed-forward network with GELU activation

Pre-norm (LayerNorm before sub-layer) is used, following the ViT paper.

In [ ]:
class TransformerBlock(nn.Module):
    """
    A single Transformer encoder block.

    Architecture:
    1. LayerNorm -> Multi-Head Self-Attention -> Residual Add
    2. LayerNorm -> MLP (Linear -> GELU -> Dropout -> Linear -> Dropout) -> Residual Add

    The MLP inner dimension is embed_dim * mlp_ratio (default 4x expansion).

    Hints:
    - Use nn.LayerNorm(embed_dim) for layer normalization
    - Use nn.MultiheadAttention(embed_dim, num_heads, dropout, batch_first=True)
    - The residual connection adds the INPUT to the OUTPUT of each sub-layer
    - Pre-norm means LayerNorm is applied BEFORE the sub-layer, not after
    - GELU activation: nn.GELU()
    """

    def __init__(self, embed_dim=768, num_heads=12, mlp_ratio=4.0, dropout=0.1):
        super().__init__()

        # TODO: Implement the following components:
        # self.ln1 = ...        (LayerNorm for attention)
        # self.attn = ...       (MultiheadAttention)
        # self.ln2 = ...        (LayerNorm for MLP)
        # self.mlp = nn.Sequential(...)  (two-layer MLP with GELU)

    def forward(self, x):
        # TODO: Implement forward pass with pre-norm and residual connections
        # 1. Apply LN1, then self-attention, then add residual
        # 2. Apply LN2, then MLP, then add residual
        pass

# Verification
block = TransformerBlock().to(device)
test_tokens = torch.randn(2, 197, 768).to(device)
block_output = block(test_tokens)
assert block_output.shape == test_tokens.shape, f"Shape mismatch: {block_output.shape}"
print(f"TransformerBlock output shape: {block_output.shape} -- correct!")

### TODO: Assemble the Full Vision Transformer

In [ ]:
class VisionTransformer(nn.Module):
    """
    The complete Vision Transformer for image classification.

    Architecture:
    1. PatchEmbedding layer (image -> sequence of patch tokens)
    2. Dropout
    3. Stack of 'depth' TransformerBlocks
    4. LayerNorm on the [CLS] token output
    5. Linear classification head ([CLS] token -> num_classes logits)

    The forward pass:
    1. Embed patches (including [CLS] token and position embeddings)
    2. Apply dropout
    3. Pass through all Transformer blocks
    4. Extract the [CLS] token (index 0 in the sequence dimension)
    5. Apply LayerNorm
    6. Pass through the classification head

    Hints:
    - Stack blocks using nn.Sequential(*[TransformerBlock(...) for _ in range(depth)])
    - The [CLS] token is at position 0: x[:, 0] extracts it from all batches
    - The classification head is a single nn.Linear(embed_dim, num_classes)
    """

    def __init__(
        self, img_size=224, patch_size=16, in_channels=3,
        num_classes=8, embed_dim=768, depth=12,
        num_heads=12, mlp_ratio=4.0, dropout=0.1
    ):
        super().__init__()

        # TODO: Implement the ViT components:
        # self.patch_embed = ...
        # self.dropout = ...
        # self.blocks = nn.Sequential(...)
        # self.ln = ...
        # self.head = ...

    def forward(self, x):
        # TODO: Implement the forward pass
        pass

# Verification
vit = VisionTransformer(num_classes=NUM_CLASSES).to(device)
test_img = torch.randn(2, 3, 224, 224).to(device)
logits = vit(test_img)
assert logits.shape == (2, NUM_CLASSES), f"Expected (2, {NUM_CLASSES}), got {logits.shape}"
total_params = sum(p.numel() for p in vit.parameters())
print(f"ViT output shape: {logits.shape} -- correct!")
print(f"Total parameters: {total_params / 1e6:.1f}M")

### Thought Questions

1. Why does ViT use a Conv2d for patch projection instead of actually flattening patches and multiplying by a weight matrix? (Hint: think about computational efficiency and memory layout.)
2. What would happen if we removed position embeddings entirely? The model would treat the image as a "bag of patches" with no spatial information. How would this affect classification of skin lesions?
3. The [CLS] token attends to all 196 patches through all 12 layers. Calculate the total number of attention operations the [CLS] token participates in across all layers and heads. Why is this important for building a holistic image representation?

---

## 3.5 Training Strategy

### Loading Pretrained Weights

Training ViT from scratch on 25K images would fail due to overfitting. We use ImageNet-21k pretrained weights from the timm library, which provides strong visual features learned from 14 million images.

In [ ]:
import timm

def create_vit_model(num_classes=8, pretrained=True):
    """
    Create a ViT-Base model with pretrained ImageNet-21k weights.
    We use timm to load pretrained weights, then replace the classification head.
    """
    model = timm.create_model(
        'vit_base_patch16_224',
        pretrained=pretrained,
        num_classes=num_classes
    )
    return model.to(device)

pretrained_vit = create_vit_model()
pretrained_params = sum(p.numel() for p in pretrained_vit.parameters())
print(f"Pretrained ViT-Base parameters: {pretrained_params / 1e6:.1f}M")

### TODO: Optimizer and Learning Rate Scheduler

In [ ]:
def create_optimizer_and_scheduler(model, lr=1e-4, weight_decay=0.05, warmup_epochs=5, total_epochs=30):
    """
    Create an AdamW optimizer and a cosine learning rate scheduler with warmup.

    Optimizer: AdamW
    - lr: Peak learning rate (reached after warmup)
    - weight_decay: 0.05 (standard for ViT fine-tuning)
    - betas: (0.9, 0.999)

    Scheduler: Linear warmup followed by cosine decay
    - Warmup: linearly increase lr from 0 to peak over warmup_epochs
    - Cosine: decay lr from peak to 0 over remaining epochs

    Why AdamW over SGD?
    - ViT training is sensitive to optimizer choice. Adam-family optimizers
      provide per-parameter adaptive learning rates, which is important because
      ViT has parameters with very different gradient scales (position embeddings
      vs. attention weights vs. MLP weights).

    Why cosine schedule with warmup?
    - Warmup prevents the randomly initialized classification head from causing
      large gradient updates that destabilize the pretrained features.
    - Cosine decay smoothly reduces the learning rate, allowing fine-grained
      optimization in later epochs.

    Hints:
    - Use torch.optim.AdamW for the optimizer
    - Use torch.optim.lr_scheduler.CosineAnnealingLR for cosine decay
    - For warmup, use torch.optim.lr_scheduler.LinearLR with start_factor=0.01
    - Chain them with torch.optim.lr_scheduler.SequentialLR

    Returns:
        optimizer, scheduler
    """
    # TODO: Implement optimizer and scheduler
    pass

optimizer, scheduler = create_optimizer_and_scheduler(pretrained_vit)
assert optimizer is not None, "Optimizer not created"
assert scheduler is not None, "Scheduler not created"
print("Optimizer and scheduler created successfully.")

### TODO: Full Training Loop

In [ ]:
def train_vit(model, train_loader, val_loader, num_epochs=30, lr=1e-4):
    """
    Full training loop for the Vision Transformer.

    For each epoch:
    1. Train on all batches (forward, loss, backward, step, scheduler step)
    2. Evaluate on validation set
    3. Track and print: train loss, val loss, val balanced accuracy, learning rate
    4. Save the best model (by validation balanced accuracy)

    Include:
    - Class-weighted cross-entropy loss
    - Gradient clipping (max_norm=1.0) to prevent exploding gradients
    - Model checkpointing (save best model state_dict)

    Why gradient clipping?
    - Self-attention can produce large gradient norms, especially in early training
      when the model is adapting pretrained features to a new domain.

    Hints:
    - Use torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    - Track best_val_balanced_acc and save model when it improves
    - Print a formatted table of metrics for each epoch
    - Return training history (lists of losses and metrics) for plotting

    Returns:
        history: Dict with keys 'train_loss', 'val_loss', 'val_balanced_acc', 'lr'
    """
    # TODO: Implement the full training loop
    pass

### Train the ViT

In [ ]:
# Train for 30 epochs (adjust num_epochs for Colab time constraints)
# On a T4 GPU, 30 epochs takes approximately 45-60 minutes
NUM_EPOCHS = 30

print(f"\nTraining ViT-Base ({NUM_EPOCHS} epochs)...")
print("=" * 65)
vit_history = train_vit(pretrained_vit, train_loader, val_loader, num_epochs=NUM_EPOCHS)

# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(vit_history['train_loss'], label='Train')
axes[0].plot(vit_history['val_loss'], label='Validation')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(vit_history['val_balanced_acc'])
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Balanced Accuracy')
axes[1].set_title('Validation Balanced Accuracy')
axes[1].grid(True, alpha=0.3)

axes[2].plot(vit_history['lr'])
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Learning Rate')
axes[2].set_title('Learning Rate Schedule')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBest validation balanced accuracy: {max(vit_history['val_balanced_acc']):.4f}")

---

## 3.6 Evaluation

Now we evaluate the trained ViT model and compare it against the ResNet-18 baseline.

### TODO: Comprehensive Evaluation

In [ ]:
def comprehensive_evaluation(model, dataloader, class_names, device):
    """
    Perform a comprehensive evaluation of the trained model.

    Generate the following:
    1. Classification report (precision, recall, F1 per class)
    2. Confusion matrix heatmap
    3. Per-class sensitivity bar chart (highlighting melanoma and other malignant classes)
    4. ROC curves for each class (one-vs-rest) with AUC values
    5. Macro-averaged AUROC

    Hints:
    - Use sklearn.metrics.classification_report with output_dict=True
    - Use sklearn.metrics.confusion_matrix and seaborn.heatmap
    - For ROC curves, you need predicted probabilities (apply softmax to logits)
    - Use sklearn.metrics.roc_curve and roc_auc_score with multi_class='ovr'
    - Create a 2x2 subplot figure for all visualizations

    Returns:
        results: Dict with 'balanced_accuracy', 'macro_auroc', 'melanoma_sensitivity',
                 'per_class_sensitivity'
    """
    # TODO: Implement comprehensive evaluation
    pass

# Evaluate the trained ViT
print("Evaluating ViT-Base...")
vit_results = comprehensive_evaluation(pretrained_vit, val_loader, CLASS_NAMES, device)

### TODO: Baseline vs. ViT Comparison

In [ ]:
def compare_baseline_vs_vit(baseline_results, vit_results, class_names):
    """
    Create a side-by-side comparison of baseline (ResNet-18) vs ViT results.

    Generate:
    1. A grouped bar chart comparing per-class sensitivity for both models
    2. A summary table with overall metrics (balanced accuracy, macro AUROC, melanoma sensitivity)
    3. Print the improvement percentage for each metric

    Hints:
    - Use matplotlib grouped bar chart with two bars per class
    - Highlight the melanoma class in a different color
    - Include percentage improvement annotations
    """
    # TODO: Implement the comparison visualization
    pass

# Run the comparison (requires baseline_results from Section 3.3)
# If you skipped baseline evaluation, evaluate the baseline first:
# baseline_results = comprehensive_evaluation(baseline_model, val_loader, CLASS_NAMES, device)
# compare_baseline_vs_vit(baseline_results, vit_results, CLASS_NAMES)

### Thought Questions

1. Which classes does the ViT improve on most compared to the CNN baseline? Is there a pattern (e.g., classes with more spatial complexity)?
2. Look at the confusion matrix. What are the most common misclassification pairs? Do these make clinical sense (e.g., melanoma confused with nevus is a known diagnostic challenge)?
3. How calibrated are the model's probability outputs? If the model predicts 80% melanoma, is the true positive rate close to 80%? Why does calibration matter for clinical deployment?

---

## 3.7 Error Analysis

Understanding where and why the model fails is as important as measuring its accuracy. Systematic error analysis reveals failure patterns that inform model improvement.

### TODO: Systematic Error Analysis

In [ ]:
def error_analysis(model, dataloader, class_names, device, num_examples=5):
    """
    Perform systematic error analysis on misclassified samples.

    Steps:
    1. Collect all misclassified samples with their predicted and true labels
    2. Group errors by (true_class, predicted_class) pair
    3. For the top 3 most frequent error pairs:
       a. Display num_examples misclassified images
       b. Show the model's full probability distribution for each
       c. Identify common visual patterns in the errors
    4. Compute error rates stratified by predicted confidence:
       - High confidence errors (predicted probability > 0.8): most dangerous
       - Medium confidence errors (0.5-0.8): potentially salvageable with thresholding
       - Low confidence errors (< 0.5): model was uncertain, appropriate for human review

    Hints:
    - Store (image, true_label, pred_label, probabilities) for all misclassified samples
    - Use Counter to find the most frequent error pairs
    - High-confidence errors are the most clinically dangerous

    Returns:
        error_summary: Dict with error counts per category and confidence stratification
    """
    # TODO: Implement error analysis
    pass

error_summary = error_analysis(pretrained_vit, val_loader, CLASS_NAMES, device)

### Thought Questions

1. Identify the top 3 failure modes. For each, propose a concrete mitigation strategy (data augmentation, architecture change, post-processing, or human-in-the-loop).
2. Are the high-confidence errors clinically dangerous? What would happen if a PCP trusted the model's confident but wrong melanoma prediction?
3. Propose a confidence-based triage system: what probability thresholds would you use for "auto-approve," "flag for dermatologist review," and "urgent referral"?

---

## 3.8 Scalability and Deployment Considerations

In production, the model must meet strict latency and throughput requirements. We benchmark inference performance to verify deployment readiness.

### TODO: Inference Benchmarking

In [ ]:
def benchmark_inference(model, img_size=224, batch_sizes=[1, 4, 16, 64], num_warmup=10, num_runs=50):
    """
    Benchmark model inference latency and throughput.

    For each batch size:
    1. Run num_warmup forward passes to warm up GPU
    2. Run num_runs timed forward passes
    3. Record: mean latency (ms), std latency, throughput (images/sec)

    Also measure:
    - Model size in MB (sum of parameter sizes)
    - Peak GPU memory usage during inference

    Hints:
    - Use torch.cuda.synchronize() before timing to ensure GPU operations complete
    - Use time.perf_counter() for high-resolution timing
    - Use torch.cuda.max_memory_allocated() for peak memory
    - Set model.eval() and use torch.no_grad()

    Print a formatted table:
    | Batch Size | Latency (ms) | Throughput (img/s) | GPU Memory (MB) |

    Returns:
        benchmarks: List of dicts with latency, throughput, memory per batch size
    """
    # TODO: Implement inference benchmarking
    pass

benchmarks = benchmark_inference(pretrained_vit)

### Thought Questions

1. Does the ViT-Base model meet the 500ms latency requirement for single-image inference on a T4 GPU? What about on a CPU?
2. How does latency scale with batch size? Is there a "sweet spot" batch size for maximizing throughput while staying within latency bounds?
3. If the model were too slow, what optimization techniques could be applied? Consider: mixed precision (FP16), knowledge distillation to a smaller model, token pruning, or dynamic resolution.

---

## 3.9 Ethical and Regulatory Analysis

Deploying an AI model in healthcare carries significant ethical and regulatory responsibilities. This section asks you to think critically about the broader implications.

### TODO: Ethical Impact Assessment

In [ ]:
def ethical_analysis():
    """
    Write a structured ethical impact assessment for deploying this skin lesion
    classifier in primary care clinics.

    Address the following (print as formatted text):

    1. BIAS AND FAIRNESS
       - Fitzpatrick skin type representation: The ISIC dataset is heavily skewed
         toward lighter skin types (Fitzpatrick I-III). How does this affect model
         performance on darker skin (Fitzpatrick IV-VI)?
       - What fairness metrics should be tracked?
       - Propose a concrete mitigation strategy.

    2. CLINICAL SAFETY
       - What is the appropriate role of the AI? (decision support vs. autonomous diagnosis)
       - What safeguards prevent over-reliance on the model by PCPs?
       - How should model uncertainty be communicated to clinicians?

    3. REGULATORY COMPLIANCE
       - FDA 510(k) requirements for software as a medical device (SaMD)
       - HIPAA compliance for patient image handling
       - Clinical validation requirements (prospective study design)

    4. FAILURE MODE RESPONSIBILITY
       - If the model misses a melanoma, who bears liability?
       - How should this be addressed in the product's terms of use?

    This is a written analysis, not code. Use print() statements with formatted text.
    """
    # TODO: Write the ethical impact assessment
    pass

ethical_analysis()

### Thought Questions

1. Should this model be deployed without separate clinical validation on darker skin types? What are the ethical implications of deploying a model with known demographic blind spots?
2. How would you design a prospective clinical trial to validate this model? What would the control arm look like?
3. If the model's false negative rate for melanoma is 8%, is it ethical to deploy it? Consider that dermatologists have a ~5% false negative rate and PCPs without AI assistance have a ~20% false negative rate.

---

## Summary

In this notebook, you built an end-to-end Vision Transformer pipeline for clinical skin lesion classification:

1. **Data Pipeline**: Loaded the ISIC 2019 dataset with augmentation and class-balanced sampling
2. **CNN Baseline**: Established a ResNet-18 baseline for comparison
3. **ViT from Scratch**: Implemented PatchEmbedding, TransformerBlock, and the full VisionTransformer class, understanding each component from first principles
4. **Transfer Learning**: Fine-tuned a pretrained ViT-Base with warmup + cosine scheduling
5. **Evaluation**: Comprehensive metrics including per-class sensitivity, ROC curves, and baseline comparison
6. **Error Analysis**: Identified failure modes and confidence-stratified error rates
7. **Deployment**: Benchmarked inference latency and throughput for production readiness
8. **Ethics**: Assessed bias, safety, and regulatory considerations

**Key takeaway**: The Vision Transformer's global self-attention mechanism enables it to capture long-range spatial relationships in dermoscopic images -- particularly important for atypical lesions where diagnostic features span distant regions of the image.

**Next steps**: Read Section 4 (Production and System Design Extension) of the accompanying case study document for a detailed look at how SkinSight AI would deploy this model in production, including API design, monitoring, A/B testing, and cost analysis.